***GENERATED CODE FOR financialdata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Channel Partners', 'max': 'Small Business', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Segment'}, {'feature_label': 'Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Canada', 'max': 'United States of America', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Product ', 'transformation_label': 'String Indexer'}], 'feature': ' Product ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' Amarilla ', 'max': ' Velo ', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Product '}, {'feature_label': ' Product ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Product ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Discount Band ', 'transformation_label': 'String Indexer'}], 'feature': ' Discount Band ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' High ', 'max': ' None ', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Discount Band '}, {'feature_label': ' Discount Band ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Discount Band ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Manufacturing Price ', 'transformation_label': 'String Indexer'}], 'feature': ' Manufacturing Price ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $10.00 ', 'max': ' $5.00 ', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Manufacturing Price '}, {'feature_label': ' Manufacturing Price ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Manufacturing Price ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Sale Price ', 'transformation_label': 'String Indexer'}], 'feature': ' Sale Price ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $12.00 ', 'max': ' $7.00 ', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Sale Price '}, {'feature_label': ' Sale Price ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Sale Price ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Gross Sales ', 'transformation_label': 'String Indexer'}], 'feature': ' Gross Sales ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $1,006,600.00 ', 'max': ' $998,200.00 ', 'missing': '0', 'distinct': '394'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Gross Sales '}, {'feature_label': ' Gross Sales ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Gross Sales ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Discounts ', 'transformation_label': 'String Indexer'}], 'feature': ' Discounts ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $-   ', 'max': ' $983.75 ', 'missing': '0', 'distinct': '355'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Discounts '}, {'feature_label': ' Discounts ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Discounts ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '  Sales ', 'transformation_label': 'String Indexer'}], 'feature': '  Sales ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $1,017,338.00 ', 'max': ' $99,758.75 ', 'missing': '0', 'distinct': '399'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '  Sales '}, {'feature_label': '  Sales ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('  Sales ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' COGS ', 'transformation_label': 'String Indexer'}], 'feature': ' COGS ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $1,101.00 ', 'max': ' $99,060.00 ', 'missing': '0', 'distinct': '391'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' COGS '}, {'feature_label': ' COGS ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' COGS ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Profit ', 'transformation_label': 'String Indexer'}], 'feature': ' Profit ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' $(1,008.75)', 'max': ' $99,814.50 ', 'missing': '0', 'distinct': '396'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Profit '}, {'feature_label': ' Profit ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Profit ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2014', 'max': '9/1/2014', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Month Name ', 'transformation_label': 'String Indexer'}], 'feature': ' Month Name ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' April ', 'max': ' September ', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Month Name '}, {'feature_label': ' Month Name ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Month Name ')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run financialdataHooks.ipynb
try:
	#sourcePreExecutionHook()

	financialsample = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Financial Sample.csv', 'filename': 'Financial Sample.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.61.134', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(financialsample)

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run financialdataHooks.ipynb
try:
	#transformationPreExecutionHook()

	financialdataautofe = TransformationMain.run(financialsample,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Segment", "transformation_label": "String Indexer"}], "feature": "Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Channel Partners", "max": "Small Business", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Segment"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Canada", "max": "United States of America", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": " Product ", "transformation_label": "String Indexer"}], "feature": " Product ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " Amarilla ", "max": " Velo ", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Product "}, {"transformationsData": [{"feature_label": " Discount Band ", "transformation_label": "String Indexer"}], "feature": " Discount Band ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " High ", "max": " None ", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Discount Band "}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Units Sold", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1594.19", "stddev": "851.68", "min": "214.0", "max": "4492.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Units Sold"}, {"transformationsData": [{"feature_label": " Manufacturing Price ", "transformation_label": "String Indexer"}], "feature": " Manufacturing Price ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $10.00 ", "max": " $5.00 ", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Manufacturing Price "}, {"transformationsData": [{"feature_label": " Sale Price ", "transformation_label": "String Indexer"}], "feature": " Sale Price ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $12.00 ", "max": " $7.00 ", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Sale Price "}, {"transformationsData": [{"feature_label": " Gross Sales ", "transformation_label": "String Indexer"}], "feature": " Gross Sales ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $1,006,600.00 ", "max": " $998,200.00 ", "missing": "0", "distinct": "394"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Gross Sales "}, {"transformationsData": [{"feature_label": " Discounts ", "transformation_label": "String Indexer"}], "feature": " Discounts ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $-   ", "max": " $983.75 ", "missing": "0", "distinct": "355"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Discounts "}, {"transformationsData": [{"feature_label": "  Sales ", "transformation_label": "String Indexer"}], "feature": "  Sales ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $1,017,338.00 ", "max": " $99,758.75 ", "missing": "0", "distinct": "399"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "  Sales "}, {"transformationsData": [{"feature_label": " COGS ", "transformation_label": "String Indexer"}], "feature": " COGS ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $1,101.00 ", "max": " $99,060.00 ", "missing": "0", "distinct": "391"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " COGS "}, {"transformationsData": [{"feature_label": " Profit ", "transformation_label": "String Indexer"}], "feature": " Profit ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " $(1,008.75)", "max": " $99,814.50 ", "missing": "0", "distinct": "396"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Profit "}, {"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2014", "max": "9/1/2014", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.91", "stddev": "3.34", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Month Number"}, {"transformationsData": [{"feature_label": " Month Name ", "transformation_label": "String Indexer"}], "feature": " Month Name ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " April ", "max": " September ", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Month Name "}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2013.75", "stddev": "0.44", "min": "2013", "max": "2014", "missing": "0"}, "updatedLabel": "Year"}]}))

	#transformationPostExecutionHook(financialdataautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run financialdataHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(financialdataautofe, ["Units Sold", "Month Number", "Year", "Country_stringindexer", " Product _stringindexer", " Discount Band _stringindexer", " Manufacturing Price _stringindexer", " Sale Price _stringindexer", " Gross Sales _stringindexer", " Discounts _stringindexer", "  Sales _stringindexer", " COGS _stringindexer", " Profit _stringindexer", "Date_stringindexer", " Month Name _stringindexer"], "Segment_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

